In [ ]:
import torch
from torch import nn

# Define the custom neural network
class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        # Define layers of the neural network
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(), # Activation function, which introduces non-linearity (prevents the vainishing gradient problem)
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),  # ReLU does not change dimensions
            nn.MaxPool2d(kernel_size=2, stride=2),  # Reduces spatial size to 112x112 --> shape: (128, 112, 112) /(n_channels, height, width)

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Reduces spatial size to 56x56 --> shape: (512, 56, 56) /(n_channels, height, width)
            nn.MaxPool2d(kernel_size=3, stride=3)  # Reduces (56, 56) → (18, 18)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),  # Converts a multi-dimensional tensor (e.g., feature maps from convolutional layers) into a 1D vector. New shape: (batch_size,512×16×16)=(batch_size,131072)
            # Flattening removes spatial dimensions, keeping only features.
            nn.Linear(in_features=512 * 18 * 18, out_features=1024),
            nn.ReLU(),
            nn.Dropout(0.5), # Randomly sets 50% of neurons to zero during training to prevent overfitting. No change in dimensions. Output shape remains: (batch_size, 1024)
            nn.Linear(in_features=1024, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=200)  # Output logits for 200 classes
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
def train(epoch, model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()

        # Compute prediction and loss
        pred = model(inputs)
        loss = criterion(pred, targets)

        # Backpropagation

        # Zero the parameter gradients (clear previous gradients)
        optimizer.zero_grad()
        # Backward pass: Compute gradients
        loss.backward()
        # Update model parameters
        optimizer.step()

        # Accumulate loss for logging
        running_loss += loss.item()

        # Compute accuracy
        _, predicted = pred.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # Print status every N batches (optional)
        if (batch_idx + 1) % 100 == 0:
            print(f'Batch {batch_idx+1}/{len(train_loader)} | Loss: {loss.item():.6f}')

    # Compute average loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total
    print(f'Train Epoch: {epoch} Loss: {train_loss:.6f} Acc: {train_accuracy:.2f}%')

    return train_loss, train_accuracy

# Define a loss function (CrossEntropyLoss)
In PyTorch, the criterion(pred, targets) (loss function) returns a single scalar tensor representing the computed loss. .item() is a PyTorch Tensor method that extracts the numerical value from a single-element tensor and converts it to a Python float.

criterion = nn.CrossEntropyLoss()
loss = criterion(pred, target)

print(loss)        # Output: tensor(0.7981, grad_fn=<NllLossBackward0>)
print(loss.item()) # Output: 0.7981389760971069 (Python float)

In [ ]:
# Validation loop
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0

    correct, total = 0, 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.cuda(), targets.cuda()

            # Forward pass: Compute predictions
            outputs = model(inputs)
            # Compute loss
            loss = criterion(outputs, targets)
            # Accumulate loss for logging
            val_loss += loss.item()

             # Compute accuracy
            _, predicted = outputs.max(1) # Get the predicted class
            total += targets.size(0) # Number of samples in batch
            correct += predicted.eq(targets).sum().item() # Count correct predictions

    # Compute average loss and accuracy
    val_loss = val_loss / len(val_loader)
    val_accuracy = 100. * correct / total

    print(f'Validation Loss: {val_loss:.6f} Acc: {val_accuracy:.2f}%')

    return val_loss, val_accuracy  # Return values for tracking progress

In [ ]:
model = CustomNet().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_acc = 0

# Run the training process for {num_epochs} epochs
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    print(f"Epoch {epoch}\n-------------------------------")
    train_loss, train_acc = train(epoch, model, train_loader, criterion, optimizer)

    # At the end of each training iteration, perform a validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion)

    # Best validation accuracy
    best_acc = max(best_acc, val_accuracy)


print(f'Best validation accuracy: {best_acc:.2f}%')
